In [27]:
import sqlite3
import numpy as np
import mysql.connector
import pandas as pd
import getpass
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
plotly.tools.set_credentials_file(username='mainakchain', api_key='B9m4DVrYYmsoW3jDSiok')
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
import plotly.figure_factory as ff
# from scipy import stats
# from sklearn.utils import shuffle
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestRegressor
# from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor
import datetime
# import holidays
from sklearn.model_selection import cross_val_score
# from sklearn.metrics import median_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.svm import LinearSVR
# from sklearn.svm import SVR
import xgboost as xgb

In [2]:
toll_time = pd.read_csv('toll_time.csv')

In [3]:
toll_time = toll_time[toll_time['origin'] == 'Jamshedpur'][toll_time['destination'] == 'HOSUR']

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [4]:
toll_time.drop(['Unnamed: 0'], axis=1, inplace=True)

In [5]:
new_toll_time = toll_time.copy()

In [6]:
today_date = datetime.datetime.now()
today_date.strftime("%Y-%m-%d %H:%M:%S")
today = today_date.strftime("%Y-%m-%d %H:%M:%S")
new_toll_time['today'] = datetime.datetime.strptime(today, '%Y-%m-%d %H:%M:%S')

In [7]:
new_toll_time['start_date'] = new_toll_time['start_date'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['days_from_now'] = new_toll_time['today'] - new_toll_time['start_date'] 
new_toll_time['days_from_now'] = new_toll_time['days_from_now'].apply(lambda x: x.days)

In [8]:
new_toll_time = toll_time.copy()

In [9]:
today_date = datetime.datetime.now()
today_date.strftime("%Y-%m-%d %H:%M:%S")
today = today_date.strftime("%Y-%m-%d %H:%M:%S")
new_toll_time['today'] = datetime.datetime.strptime(today, '%Y-%m-%d %H:%M:%S')

new_toll_time['start_date'] = new_toll_time['start_date'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['days_from_now'] = new_toll_time['today'] - new_toll_time['start_date'] 
new_toll_time['days_from_now'] = new_toll_time['days_from_now'].apply(lambda x: x.days)

new_toll_time['unloading_in_time'] = new_toll_time['unloading_in_time'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['avg_travel_time'] = new_toll_time['start_date'] + (new_toll_time['unloading_in_time'] - new_toll_time['start_date'])/2

new_toll_time['month_of_travel'] = new_toll_time['avg_travel_time'].apply(lambda x: x.month)

new_toll_time['ist_timestamp'] = new_toll_time['ist_timestamp'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
new_toll_time['time_of_travel'] = new_toll_time.ist_timestamp.apply(lambda x: x.hour)

new_toll_time['day_of_travel'] = new_toll_time['avg_travel_time'].apply(lambda x: x.weekday())

new_toll_time['loading_out_time'] = new_toll_time['loading_out_time'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_toll_time['loading_out_time'] = new_toll_time['loading_out_time'].apply(lambda x: x.hour)

In [10]:
new_toll_time = new_toll_time[['toll_both','vehicle_no','loading_out_time','distance_travelled','time_of_travel','day_of_travel','month_of_travel','days_from_now','time_taken']]

In [11]:
train_df = new_toll_time.copy()
test_df= pd.DataFrame(columns=['toll_both','vehicle_no','loading_out_time', 'distance_travelled', 'time_of_travel',
                                         'day_of_travel', 'month_of_travel', 'days_from_now'])
choice = True
while(choice):
    today_date = datetime.datetime.now()
    vehicle = raw_input('Enter the vehicle no.: ')
    start = raw_input('Enter the start date in %Y-%m-%d %H:%M:%S format: ')
    start_delta = datetime.datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
    
    loading_out = raw_input('Enter the loading out time in %Y-%m-%d %H:%M:%S format: ')
    loading_out_delta = datetime.datetime.strptime(loading_out, '%Y-%m-%d %H:%M:%S')
    
    a = []
    loop = input('Enter the number of tolls vehicle crossed sequentitially: ')
    count = 0
    print '      '
    while (count != loop):
        b = []
        toll = raw_input('Toll both: ')
        distance = input('Distance travelled from 1st toll in km: ')
        ist = raw_input('IST Timestamp in %Y-%m-%d %H:%M:%S format: ')
        ist_delta = datetime.datetime.strptime(ist, '%Y-%m-%d %H:%M:%S')
        b.extend([toll, vehicle, loading_out_delta.hour, distance, ist_delta.hour, ist_delta.weekday(), ist_delta.month,
                      (today_date - loading_out_delta).days, (ist_delta - loading_out_delta).total_seconds()/3600])
        a.append(b)
        count+=1
    
    add_df = pd.DataFrame(a, columns= ['toll_both','vehicle_no','loading_out_time', 'distance_travelled', 'time_of_travel',
                                         'day_of_travel', 'month_of_travel', 'days_from_now', 'time_taken'])
    train_df = train_df.append(add_df)
    
    test_toll = raw_input('Enter the numbers of tolls the vehicle shall take in the journey sequentially: ')
    test_toll_list = test_toll.split(',')
    test_toll_list = [str(a) for a in test_toll_list]

    a = []

    for toll_val in test_toll_list:
        b = []
        distance = toll_time['distance_travelled'][toll_time.toll_both == toll_val].median()
        ist_delta = loading_out_delta + timedelta(hours=toll_time['time_taken'][toll_time.toll_both == toll_val].median())
        b.extend([toll_val, vehicle, loading_out_delta.hour, distance, ist_delta.hour, ist_delta.weekday(), ist_delta.month,
                      (today_date - loading_out_delta).days])

        a.append(b)
    add_test_df = pd.DataFrame(a, columns= ['toll_both','vehicle_no','loading_out_time', 'distance_travelled', 'time_of_travel',
                                         'day_of_travel', 'month_of_travel', 'days_from_now'])
    
    test_df = test_df.append(add_test_df)
    
    c = raw_input('Want to enter more vehicles?(y/n)')
    
    if (c == 'y' or c == 'Y'):
        continue
    elif (c == 'n' or c == 'N'):
        choice = False
        print 'bbye'
    else:
        print 'Wrong Choice! Exiting'
        choice = False

KeyboardInterrupt: 

In [12]:
train_df = pd.read_csv('final_2nd_train_df.csv')
test_df = pd.read_csv('final_2nd_test_df.csv')

In [13]:
train_df.tail(10)

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,time_taken
988,33,9,14,1165.77,9,4,12,15,90.697222
989,33,33,23,1242.01,20,0,11,38,189.460833
990,27,45,12,0.00,8,5,11,40,20.335556
991,25,45,12,95.10,10,5,11,40,22.658056
992,20,45,12,146.80,12,5,11,40,24.124722
993,27,46,21,0.00,17,6,10,52,20.183611
994,25,46,21,94.60,19,6,10,52,22.575278
995,20,46,21,146.50,21,6,10,52,24.169722
996,9,46,21,204.60,23,6,10,52,26.326389
997,10,46,21,293.59,1,0,10,52,28.585278


In [14]:
test_df

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now
0,1,1,12,203.320,10,6,11,40
1,2,1,12,291.250,13,6,11,40
2,3,1,12,375.245,20,6,11,40
3,4,1,12,490.530,0,0,11,40
4,5,1,12,517.415,3,0,11,40
5,6,1,12,556.620,5,0,11,40
6,7,1,12,556.620,5,0,11,40
7,8,1,12,684.270,11,0,11,40
8,9,1,12,753.930,16,0,11,40
9,10,1,12,897.510,2,1,11,40


In [15]:
train_df.shape, test_df.shape

((998, 9), (38, 8))

In [16]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [17]:
def change_toll_booth_to_dummy(df):
    num = 1
    dict_toll_both = {}
    for toll in df['toll_both'].unique().tolist():
        df['toll_both'] = df['toll_both'].replace({toll: num})
        dict_toll_both[num] = toll
        num += 1
    return dict_toll_both

In [18]:
dict_toll_both_train_df = change_toll_booth_to_dummy(train_df)
dict_toll_both_test_df = change_toll_booth_to_dummy(test_df)

In [19]:
def change_vehicle_no_to_dummy(df):
    num = 1
    dict_vehicle_no = {}
    for truck in df['vehicle_no'].unique().tolist():
        df['vehicle_no'] = df['vehicle_no'].replace({truck: num})
        dict_vehicle_no[num] = truck
        num += 1
    return dict_vehicle_no

In [20]:
dict_vehicle_no_train_df = change_vehicle_no_to_dummy(train_df)
dict_vehicle_no_test_df = change_vehicle_no_to_dummy(test_df)

In [163]:
# MODEL BUILDING AND APPLICATION

In [23]:
train_X = train_df.drop(['time_taken'], axis=1)
train_y = train_df.time_taken

In [24]:
train_X.shape

(998, 8)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_X,train_y, test_size=0.3, random_state=42)

In [66]:
scale= MinMaxScaler()
X_train_scaled = scale.fit_transform(train_X)
X_test_scaled = scale.fit_transform(test_df)

In [67]:
train_X.shape

(998, 8)

In [68]:
weights = np.ones(8)
weights[3] = weights[3]*5
weights

array([ 1.,  1.,  1.,  5.,  1.,  1.,  1.,  1.])

In [74]:
dtrain = xgb.DMatrix(X_train_scaled, label=train_y)
dtest = xgb.DMatrix(X_test_scaled)

In [108]:
# bst = bst = xgb.train(para, dtrain,15)
xgb_model = xgb.XGBRegressor(max_depth=10, min_child_weight=5, n_estimators=5000,objective='reg:linear', seed=1337, silent=0, subsample=0.9, learning_rate=0.1, colsample_bytree=0.8, nthread=-1, scale_pos_weight=2.2)

In [109]:
xgb_model.fit(X_train_scaled, train_y)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=5, missing=None, n_estimators=5000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=2.2, seed=1337, silent=0, subsample=0.9)

In [110]:
# pred = bst.predict(dtest)
pred = xgb_model.predict(X_test_scaled)

In [111]:
pred

array([  48.21331787,   49.39078903,   57.48572922,  100.44218445,
        107.0233078 ,   95.85424042,   95.99355316,  104.97574615,
        105.78533936,  109.94863129,  109.46266174,  104.82061768,
        101.19426727,  104.93113708,  115.10979462,  115.81002808,
        114.11766815,  116.41475677,  123.09128571,  129.71054077,
         20.09045601,   52.39149857,   57.60975266,   56.78306198,
         57.18423462,   69.02258301,   61.46611404,   80.10572052,
         81.07215118,   81.42498779,   77.54833221,   81.68404388,
         87.48851013,   88.98529053,  113.38775635,  112.00872803,
        115.99562836,  114.24333954], dtype=float32)

In [ ]:
xgb

In [154]:
reg_mlp.fit(X_train_scaled, y_train)
reg_mlp.score(X_test_scaled, y_test) * 100

90.5241146301952

In [155]:
scores = cross_val_score(reg_mlp, X_train_scaled, y_train, scoring='r2', n_jobs=-1)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.83 (+/- 0.03)


In [183]:
predictions = reg_mlp.predict(X_predict_scaled)

In [184]:
show_df = test_df.copy()
show_df['predicted'] = predictions

In [190]:
show_df['predict_diff'] = show_df['predicted'].diff()

In [192]:
def med_toll_time_taken(toll):
    return train_df['time_taken'][train_df['toll_both'] == toll ].median()

In [199]:
def mean_toll_time_taken(toll):
    return train_df['time_taken'][train_df['toll_both'] == toll ].mean()

In [200]:
show_df['med_time'] = show_df['toll_both']
show_df['med_time'] = show_df.med_time.apply(lambda x:med_toll_time_taken(x))
show_df['mean_time'] = show_df['toll_both']
show_df['mean_time'] = show_df.mean_time.apply(lambda x:mean_toll_time_taken(x))

In [201]:
train_df.tail(10)

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,time_taken
988,33,9,14,1165.77,9,4,12,15,90.697222
989,33,33,23,1242.01,20,0,11,38,189.460833
990,27,45,12,0.00,8,5,11,40,20.335556
991,25,45,12,95.10,10,5,11,40,22.658056
992,20,45,12,146.80,12,5,11,40,24.124722
993,27,46,21,0.00,17,6,10,52,20.183611
994,25,46,21,94.60,19,6,10,52,22.575278
995,20,46,21,146.50,21,6,10,52,24.169722
996,9,46,21,204.60,23,6,10,52,26.326389
997,10,46,21,293.59,1,0,10,52,28.585278


In [202]:
show_df

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,predicted,predict_diff,med_time,mean_time
0,1,1,12,203.320,10,6,11,40,60.305124,NaN,84.090278,84.090278
1,2,1,12,291.250,13,6,11,40,62.938448,2.633323,93.539722,112.452738
2,3,1,12,375.245,20,6,11,40,64.313343,1.374895,56.755556,61.037500
3,4,1,12,490.530,0,0,11,40,58.601461,-5.711881,92.291944,99.164912
4,5,1,12,517.415,3,0,11,40,59.647295,1.045834,106.137500,113.508902
5,6,1,12,556.620,5,0,11,40,61.671978,2.024683,63.638889,66.576692
6,7,1,12,556.620,5,0,11,40,61.477310,-0.194668,48.963056,48.963056
7,8,1,12,684.270,11,0,11,40,68.672551,7.195241,122.277500,130.865833
8,9,1,12,753.930,16,0,11,40,72.610093,3.937542,45.996250,50.308219
9,10,1,12,897.510,2,1,11,40,65.359971,-7.250122,49.686667,52.731903
